In [1]:
from ast import *

In [2]:
def dump(node, annotate_fields=True, include_attributes=False, indent='  '):
    """
    Return a formatted dump of the tree in *node*.  This is mainly useful for
    debugging purposes.  The returned string will show the names and the values
    for fields.  This makes the code impossible to evaluate, so if evaluation is
    wanted *annotate_fields* must be set to False.  Attributes such as line
    numbers and column offsets are not dumped by default.  If this is wanted,
    *include_attributes* can be set to True.
    """
    def _format(node, level=0):
        if isinstance(node, AST):
            fields = [(a, _format(b, level)) for a, b in iter_fields(node)]
            rv = '%s(%s' % (node.__class__.__name__, ', '.join(
                ('%s=%s' % field for field in fields)
                if annotate_fields else
                (b for a, b in fields)
            ))
            if include_attributes and node._attributes:
                rv += fields and ', ' or ' '
                rv += ', '.join('%s=%s' % (a, _format(getattr(node, a)))
                                for a in node._attributes)
            return rv + ')'
        elif isinstance(node, list):
            lines = ['[']
            lines.extend((indent * (level + 2) + _format(x, level + 2) + ','
                          for x in node))
            if len(lines) > 1:
                lines.append(indent * (level + 1) + ']')
            else:
                lines[-1] += ']'
            return '\n'.join(lines)
        return repr(node)
    if not isinstance(node, AST):
        raise TypeError('expected AST, got %r' % node.__class__.__name__)
    return _format(node)

def parseprint(source, filename='<unknown>', mode="exec", **kwargs):
    """Parse the source and pretty-print the AST."""
    node = parse(source, filename, mode=mode)
    print(dump(node, **kwargs))
    

# print a single expression
def ppe(exp, filename='<unknown>', mode="exec", **kwargs):
    node = parse(exp, filename, mode=mode).body[0]
    print(dump(node, **kwargs))


## Literals

In [3]:
ppe('9')

Expr(value=Num(n=9))


In [4]:
ppe('"hello"')

Expr(value=Str(s='hello'))


In [5]:
name = 'Tom'
ppe('f"hello {name}"')

Expr(value=JoinedStr(values=[
    Str(s='hello '),
    FormattedValue(value=Name(id='name', ctx=Load()), conversion=-1, format_spec=None),
  ]))


In [6]:
ppe('b"hi"')

Expr(value=Bytes(s=b'hi'))


In [7]:
ppe('[1, 2]')

Expr(value=List(elts=[
    Num(n=1),
    Num(n=2),
  ], ctx=Load()))


In [8]:
ppe('(1, 2)')

Expr(value=Tuple(elts=[
    Num(n=1),
    Num(n=2),
  ], ctx=Load()))


In [9]:
ppe('{1, 2}')

Expr(value=Set(elts=[
    Num(n=1),
    Num(n=2),
  ]))


In [10]:
ppe('{"name": "Tom", "age": 3}')

Expr(value=Dict(keys=[
    Str(s='name'),
    Str(s='age'),
  ], values=[
    Str(s='Tom'),
    Num(n=3),
  ]))


In [11]:
ppe('...')

Expr(value=Ellipsis())


## Variables

In [16]:
# Load
ppe('Tom')

Expr(value=Name(id='Tom', ctx=Load()))


In [17]:
# Store
ppe('a = 1')

Assign(targets=[
    Name(id='a', ctx=Store()),
  ], value=Num(n=1))


In [18]:
# Del
ppe('del a')

Delete(targets=[
    Name(id='a', ctx=Del()),
  ])


In [19]:
# Starred -> *b
ppe('a, *b = c')

Assign(targets=[
    Tuple(elts=[
        Name(id='a', ctx=Store()),
        Starred(value=Name(id='b', ctx=Store()), ctx=Store()),
      ], ctx=Store()),
  ], value=Name(id='c', ctx=Load()))


## Expressions

In [25]:
# Expr/UnaryOp -> 一元运算
ppe('-a')

ppe('+a')

ppe('not a')

ppe('~a')

Expr(value=UnaryOp(op=USub(), operand=Name(id='a', ctx=Load())))
Expr(value=UnaryOp(op=UAdd(), operand=Name(id='a', ctx=Load())))
Expr(value=UnaryOp(op=Not(), operand=Name(id='a', ctx=Load())))
Expr(value=UnaryOp(op=Invert(), operand=Name(id='a', ctx=Load())))


In [29]:
# BinOp -> 二元运算
ppe('a + b')
ppe('a - b')
ppe('a * b')
ppe('a / b')
ppe('a // b')
ppe('a % b')
ppe('a ** b')
ppe('a << b')
ppe('a >> b')
ppe('a | b')
ppe('a & b')
ppe('a ^ b')
ppe('a @ b')

Expr(value=BinOp(left=Name(id='a', ctx=Load()), op=Add(), right=Name(id='b', ctx=Load())))
Expr(value=BinOp(left=Name(id='a', ctx=Load()), op=Sub(), right=Name(id='b', ctx=Load())))
Expr(value=BinOp(left=Name(id='a', ctx=Load()), op=Mult(), right=Name(id='b', ctx=Load())))
Expr(value=BinOp(left=Name(id='a', ctx=Load()), op=Div(), right=Name(id='b', ctx=Load())))
Expr(value=BinOp(left=Name(id='a', ctx=Load()), op=FloorDiv(), right=Name(id='b', ctx=Load())))
Expr(value=BinOp(left=Name(id='a', ctx=Load()), op=Mod(), right=Name(id='b', ctx=Load())))
Expr(value=BinOp(left=Name(id='a', ctx=Load()), op=Pow(), right=Name(id='b', ctx=Load())))
Expr(value=BinOp(left=Name(id='a', ctx=Load()), op=LShift(), right=Name(id='b', ctx=Load())))
Expr(value=BinOp(left=Name(id='a', ctx=Load()), op=RShift(), right=Name(id='b', ctx=Load())))
Expr(value=BinOp(left=Name(id='a', ctx=Load()), op=BitOr(), right=Name(id='b', ctx=Load())))
Expr(value=BinOp(left=Name(id='a', ctx=Load()), op=BitAnd(), right=Name(id='

In [31]:
# BoolOp -> 布尔运算
ppe('a and b')
ppe('a or b')
ppe('a and b or c')

Expr(value=BoolOp(op=And(), values=[
    Name(id='a', ctx=Load()),
    Name(id='b', ctx=Load()),
  ]))
Expr(value=BoolOp(op=Or(), values=[
    Name(id='a', ctx=Load()),
    Name(id='b', ctx=Load()),
  ]))
Expr(value=BoolOp(op=Or(), values=[
    BoolOp(op=And(), values=[
        Name(id='a', ctx=Load()),
        Name(id='b', ctx=Load()),
      ]),
    Name(id='c', ctx=Load()),
  ]))


In [32]:
# Compare -> 比较
ppe('a == b')
ppe('a != b')
ppe('a < b')
ppe('a <= b')
ppe('a > b')
ppe('a >= b')
ppe('a is b')
ppe('a is not b')
ppe('a in b')
ppe('a not in b')

Expr(value=Compare(left=Name(id='a', ctx=Load()), ops=[
    Eq(),
  ], comparators=[
    Name(id='b', ctx=Load()),
  ]))
Expr(value=Compare(left=Name(id='a', ctx=Load()), ops=[
    NotEq(),
  ], comparators=[
    Name(id='b', ctx=Load()),
  ]))
Expr(value=Compare(left=Name(id='a', ctx=Load()), ops=[
    Lt(),
  ], comparators=[
    Name(id='b', ctx=Load()),
  ]))
Expr(value=Compare(left=Name(id='a', ctx=Load()), ops=[
    LtE(),
  ], comparators=[
    Name(id='b', ctx=Load()),
  ]))
Expr(value=Compare(left=Name(id='a', ctx=Load()), ops=[
    Gt(),
  ], comparators=[
    Name(id='b', ctx=Load()),
  ]))
Expr(value=Compare(left=Name(id='a', ctx=Load()), ops=[
    GtE(),
  ], comparators=[
    Name(id='b', ctx=Load()),
  ]))
Expr(value=Compare(left=Name(id='a', ctx=Load()), ops=[
    Is(),
  ], comparators=[
    Name(id='b', ctx=Load()),
  ]))
Expr(value=Compare(left=Name(id='a', ctx=Load()), ops=[
    IsNot(),
  ], comparators=[
    Name(id='b', ctx=Load()),
  ]))
Expr(value=Compare(left=

In [33]:
# Call -> 函数调用
# keyword
ppe('add(a, b=2, *c, **d)')

Expr(value=Call(func=Name(id='add', ctx=Load()), args=[
    Name(id='a', ctx=Load()),
    Starred(value=Name(id='c', ctx=Load()), ctx=Load()),
  ], keywords=[
    keyword(arg='b', value=Num(n=2)),
    keyword(arg=None, value=Name(id='d', ctx=Load())),
  ]))


In [35]:
# IfExp
ppe('a if b else c')

Expr(value=IfExp(test=Name(id='b', ctx=Load()), body=Name(id='a', ctx=Load()), orelse=Name(id='c', ctx=Load())))


In [38]:
# Attribute
ppe('person.name')

Expr(value=Attribute(value=Name(id='person', ctx=Load()), attr='name', ctx=Load()))


## Subscripting

In [42]:
# Subscript(value, slice, ctx)
# Index, Slice, ExtSlice
ppe('todos[1]')
ppe('todos[1:]')
ppe('todos[1:2, 2]')

Expr(value=Subscript(value=Name(id='todos', ctx=Load()), slice=Index(value=Num(n=1)), ctx=Load()))
Expr(value=Subscript(value=Name(id='todos', ctx=Load()), slice=Slice(lower=Num(n=1), upper=None, step=None), ctx=Load()))
Expr(value=Subscript(value=Name(id='todos', ctx=Load()), slice=ExtSlice(dims=[
    Slice(lower=Num(n=1), upper=Num(n=2), step=None),
    Index(value=Num(n=2)),
  ]), ctx=Load()))


## Comprehensions

In [44]:
# 列表、字典、集合、生成器生成式
# comprehension
ppe('[i for i in range(3)]')

Expr(value=ListComp(elt=Name(id='i', ctx=Load()), generators=[
    comprehension(target=Name(id='i', ctx=Store()), iter=Call(func=Name(id='range', ctx=Load()), args=[
        Num(n=3),
      ], keywords=[]), ifs=[], is_async=0),
  ]))


In [47]:
ppe('{key: value for key in keys for value in values}')

Expr(value=DictComp(key=Name(id='key', ctx=Load()), value=Name(id='value', ctx=Load()), generators=[
    comprehension(target=Name(id='key', ctx=Store()), iter=Name(id='keys', ctx=Load()), ifs=[], is_async=0),
    comprehension(target=Name(id='value', ctx=Store()), iter=Name(id='values', ctx=Load()), ifs=[], is_async=0),
  ]))


In [49]:
ppe('{k for k in li if k > 0}')

Expr(value=SetComp(elt=Name(id='k', ctx=Load()), generators=[
    comprehension(target=Name(id='k', ctx=Store()), iter=Name(id='li', ctx=Load()), ifs=[
        Compare(left=Name(id='k', ctx=Load()), ops=[
            Gt(),
          ], comparators=[
            Num(n=0),
          ]),
      ], is_async=0),
  ]))


In [50]:
ppe('(i for i in range(9))')

Expr(value=GeneratorExp(elt=Name(id='i', ctx=Load()), generators=[
    comprehension(target=Name(id='i', ctx=Store()), iter=Call(func=Name(id='range', ctx=Load()), args=[
        Num(n=9),
      ], keywords=[]), ifs=[], is_async=0),
  ]))


## Statements

In [57]:
# Assign
ppe('a = 1')
ppe('a = b = 1')
ppe('a, b = c')

Assign(targets=[
    Name(id='a', ctx=Store()),
  ], value=Num(n=1))
Assign(targets=[
    Name(id='a', ctx=Store()),
    Name(id='b', ctx=Store()),
  ], value=Num(n=1))
Assign(targets=[
    Tuple(elts=[
        Name(id='a', ctx=Store()),
        Name(id='b', ctx=Store()),
      ], ctx=Store()),
  ], value=Name(id='c', ctx=Load()))


In [56]:
# AnnAssign
ppe('a: int = 1')

AnnAssign(target=Name(id='a', ctx=Store()), annotation=Name(id='int', ctx=Load()), value=Num(n=1), simple=1)


In [58]:
# AugAssign
ppe('a += 1')
ppe('a -= 1')

AugAssign(target=Name(id='a', ctx=Store()), op=Add(), value=Num(n=1))
AugAssign(target=Name(id='a', ctx=Store()), op=Sub(), value=Num(n=1))


In [60]:
# Raise
ppe('raise a')

Raise(exc=Name(id='a', ctx=Load()), cause=None)


In [61]:
# Assert
ppe('assert 1 == 2')

Assert(test=Compare(left=Num(n=1), ops=[
    Eq(),
  ], comparators=[
    Num(n=2),
  ]), msg=None)


In [62]:
# Delete
ppe('del a')

Delete(targets=[
    Name(id='a', ctx=Del()),
  ])


In [63]:
# Pass
ppe('pass')

Pass()


## Imports

In [65]:
ppe('import numpy')

Import(names=[
    alias(name='numpy', asname=None),
  ])


In [66]:
ppe('from numpy import matrix')

ImportFrom(module='numpy', names=[
    alias(name='matrix', asname=None),
  ], level=0)


In [67]:
# alias
ppe('import numpy as np')

Import(names=[
    alias(name='numpy', asname='np'),
  ])


## Control flow

In [73]:
# If -> (test, body, orelse)
ppe('''if a > 0: 
    pass
elif a == 0:
    pass
else:
    pass
''')

If(test=Compare(left=Name(id='a', ctx=Load()), ops=[
    Gt(),
  ], comparators=[
    Num(n=0),
  ]), body=[
    Pass(),
  ], orelse=[
    If(test=Compare(left=Name(id='a', ctx=Load()), ops=[
        Eq(),
      ], comparators=[
        Num(n=0),
      ]), body=[
        Pass(),
      ], orelse=[
        Pass(),
      ]),
  ])


In [74]:
# For -> (target, iter, body, orelse, type_comment)
ppe('''for i in range(10):
    pass
else:
    pass
''')

For(target=Name(id='i', ctx=Store()), iter=Call(func=Name(id='range', ctx=Load()), args=[
    Num(n=10),
  ], keywords=[]), body=[
    Pass(),
  ], orelse=[
    Pass(),
  ])


In [76]:
# While -> (test, body, orelse)
ppe('''while True:
    pass
else:
    pass
''')

While(test=NameConstant(value=True), body=[
    Pass(),
  ], orelse=[
    Pass(),
  ])


In [77]:
# Break, Continue
ppe('break')
ppe('continue')

Break()
Continue()


In [81]:
# Try > ExceptHandler
ppe('''try:
    pass
except E as e:
    pass
finally:
    pass

''')

Try(body=[
    Pass(),
  ], handlers=[
    ExceptHandler(type=Name(id='E', ctx=Load()), name='e', body=[
        Pass(),
      ]),
  ], orelse=[], finalbody=[
    Pass(),
  ])


In [83]:
# With > withitem
ppe('with x, y: pass')

With(items=[
    withitem(context_expr=Name(id='x', ctx=Load()), optional_vars=None),
    withitem(context_expr=Name(id='y', ctx=Load()), optional_vars=None),
  ], body=[
    Pass(),
  ])


## Function and class definitions

In [94]:
# FunctionDef -> (name, args, body, decorator_list, returns, type_comment)
# arguments
ppe('''def add(x, y):
    return x + y
''')

ppe('''@log
def add(x: int, y: int) -> int:
    pass
''')

FunctionDef(name='add', args=arguments(args=[
    arg(arg='x', annotation=None),
    arg(arg='y', annotation=None),
  ], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[
    Return(value=BinOp(left=Name(id='x', ctx=Load()), op=Add(), right=Name(id='y', ctx=Load()))),
  ], decorator_list=[], returns=None)
FunctionDef(name='add', args=arguments(args=[
    arg(arg='x', annotation=Name(id='int', ctx=Load())),
    arg(arg='y', annotation=Name(id='int', ctx=Load())),
  ], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[
    Pass(),
  ], decorator_list=[
    Name(id='log', ctx=Load()),
  ], returns=Name(id='int', ctx=Load()))


In [86]:
# Lambda
ppe('lambda x: x+1')

Expr(value=Lambda(args=arguments(args=[
    arg(arg='x', annotation=None),
  ], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=BinOp(left=Name(id='x', ctx=Load()), op=Add(), right=Num(n=1))))


In [87]:
# Return
ppe('return 9')

Return(value=Num(n=9))


In [89]:
# Yield
ppe('yield 9')

Expr(value=Yield(value=Num(n=9)))


In [91]:
# Global
ppe('global a')

Global(names=[
    'a',
  ])


In [92]:
# Nonlocal
ppe('nonlocal a')

Nonlocal(names=[
    'a',
  ])


In [93]:
# ClassDef -> (name, bases, keywords, starargs, kwargs, body, decorator_list)
ppe('''class Dog(Animal, metaclass=meta):
    pass''')

ClassDef(name='Dog', bases=[
    Name(id='Animal', ctx=Load()),
  ], keywords=[
    keyword(arg='metaclass', value=Name(id='meta', ctx=Load())),
  ], body=[
    Pass(),
  ], decorator_list=[])


## Async and await

In [95]:
# AsyncFunctionDef -> (name, args, body, decorator_list, returns, type_comment)
ppe('''async def wait():
    await me()''')

AsyncFunctionDef(name='wait', args=arguments(args=[], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[
    Expr(value=Await(value=Call(func=Name(id='me', ctx=Load()), args=[], keywords=[]))),
  ], decorator_list=[], returns=None)


In [96]:
# Await
ppe('await 9')

Expr(value=Await(value=Num(n=9)))


In [97]:
# Async For
ppe('''async for i in range(10):
    pass''')

AsyncFor(target=Name(id='i', ctx=Store()), iter=Call(func=Name(id='range', ctx=Load()), args=[
    Num(n=10),
  ], keywords=[]), body=[
    Pass(),
  ], orelse=[])


In [98]:
# Async With
ppe('''async with me(): pass''')

AsyncWith(items=[
    withitem(context_expr=Call(func=Name(id='me', ctx=Load()), args=[], keywords=[]), optional_vars=None),
  ], body=[
    Pass(),
  ])


## Top level nodes

In [99]:
# Module
parseprint('')

Module(body=[])
